In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import resample


# Load the dataset
file_path = 'filtered_normalized_data.csv'
df = pd.read_csv(file_path)

# Assign the binary diabetes label
negative = df[df['Diabetes_012'] == 0]
positive = df[df['Diabetes_012'] == 2]

# Get the smallest class size
min_class_size = min(len(negative), len(positive))

# Resample each class to have the same size as the smallest class
negative_downsampled = resample(negative, replace=False, n_samples=min_class_size, random_state=42)
positive_downsampled = resample(positive, replace=False, n_samples=min_class_size, random_state=42)

# Concatenate all downsampled classes to create a balanced dataset
downsampled_data = pd.concat([negative_downsampled, positive_downsampled])

# The downsized dataset conains 70692 rows

X = downsampled_data.drop('Diabetes_012', axis=1).values
y = downsampled_data['Diabetes_012'].values

#k-fold k=5
kf = KFold(n_splits=5, random_state=445, shuffle=True)
n_splits=5

# Create X_full and y_full from the full dataset (Added)
X_full = df.drop('Diabetes_012', axis=1).values
y_full = df['Diabetes_012'].values



In [3]:
#Random Forest
clf = RandomForestClassifier(n_estimators=100)

avg_precision = {0: 0, 2: 0}
avg_recall = {0: 0, 2: 0}
avg_f1_score = {0: 0, 2: 0}
# Initialize accumulators for the full dataset
avg_precision_full = {0: 0, 2: 0}
avg_recall_full = {0: 0, 2: 0}
avg_f1_score_full = {0: 0, 2: 0}

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    # Calculate metrics
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    class_to_index = {label: index for index, label in enumerate(sorted(np.unique(y_test)))}

    # Accumulate metrics for averaging
    for i in [0, 2]:
        idx = class_to_index[i]
        avg_precision[i] += precision[idx]
        avg_recall[i] += recall[idx]
        avg_f1_score[i] += f1_score[idx]
    #FULL added
    y_pred_full = clf.predict(X_full)
    # Calculate metrics for the full dataset
    precision_full, recall_full, f1_score_full, _ = precision_recall_fscore_support(y_full, y_pred_full, average=None)
    class_to_index_full = {label: index for index, label in enumerate(sorted(np.unique(y_full)))}

    # Accumulate metrics for averaging (full dataset)
    for i in [0, 2]:
        idx_full = class_to_index_full[i]
        avg_precision_full[i] += precision_full[idx_full]
        avg_recall_full[i] += recall_full[idx_full]
        avg_f1_score_full[i] += f1_score_full[idx_full]

# Calculate average metrics
for i in [0, 2]:
    avg_precision[i] /= n_splits
    avg_recall[i] /= n_splits
    avg_f1_score[i] /= n_splits
# Calculate average metrics for the full dataset
for i in [0, 2]:
    avg_precision_full[i] /= n_splits
    avg_recall_full[i] /= n_splits
    avg_f1_score_full[i] /= n_splits

# Print average metrics
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-Score:", avg_f1_score)

# Print average metrics for the full dataset
print("Average Precision on Full Dataset:", avg_precision_full)
print("Average Recall on Full Dataset:", avg_recall_full)
print("Average F1-Score on Full Dataset:", avg_f1_score_full)


              precision    recall  f1-score   support

           0       0.73      0.69      0.71      7001
           2       0.71      0.75      0.73      7138

    accuracy                           0.72     14139
   macro avg       0.72      0.72      0.72     14139
weighted avg       0.72      0.72      0.72     14139
              precision    recall  f1-score   support

           0       0.74      0.69      0.72      7067
           2       0.71      0.76      0.74      7072

    accuracy                           0.73     14139
   macro avg       0.73      0.73      0.73     14139
weighted avg       0.73      0.73      0.73     14139
              precision    recall  f1-score   support

           0       0.74      0.68      0.71      7125
           2       0.70      0.76      0.73      7013

    accuracy                           0.72     14138
   macro avg       0.72      0.72      0.72     14138
weighted avg       0.72      0.72      0.72     14138
              precisio

In [4]:
#Logistic regression
clf = LogisticRegression(max_iter=3000, solver='lbfgs')

avg_precision = {0: 0, 2: 0}
avg_recall = {0: 0, 2: 0}
avg_f1_score = {0: 0, 2: 0}
# Initialize accumulators for the full dataset
avg_precision_full = {0: 0, 2: 0}
avg_recall_full = {0: 0, 2: 0}
avg_f1_score_full = {0: 0, 2: 0}

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    # Calculate metrics
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=1)
    class_to_index = {label: index for index, label in enumerate(sorted(np.unique(y_test)))}

    # Accumulate metrics for averaging
    for i in [0, 2]:
        idx = class_to_index[i]
        avg_precision[i] += precision[idx]
        avg_recall[i] += recall[idx]
        avg_f1_score[i] += f1_score[idx]
    #FULL added
    y_pred_full = clf.predict(X_full)
    # Calculate metrics for the full dataset
    precision_full, recall_full, f1_score_full, _ = precision_recall_fscore_support(y_full, y_pred_full, average=None)
    class_to_index_full = {label: index for index, label in enumerate(sorted(np.unique(y_full)))}

    # Accumulate metrics for averaging (full dataset)
    for i in [0, 2]:
        idx_full = class_to_index_full[i]
        avg_precision_full[i] += precision_full[idx_full]
        avg_recall_full[i] += recall_full[idx_full]
        avg_f1_score_full[i] += f1_score_full[idx_full]

# Calculate average metrics
for i in [0, 2]:
    avg_precision[i] /= n_splits
    avg_recall[i] /= n_splits
    avg_f1_score[i] /= n_splits
# Calculate average metrics for the full dataset
for i in [0, 2]:
    avg_precision_full[i] /= n_splits
    avg_recall_full[i] /= n_splits
    avg_f1_score_full[i] /= n_splits

# Print average metrics
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-Score:", avg_f1_score)

# Print average metrics for the full dataset
print("Average Precision on Full Dataset:", avg_precision_full)
print("Average Recall on Full Dataset:", avg_recall_full)
print("Average F1-Score on Full Dataset:", avg_f1_score_full)


              precision    recall  f1-score   support

           0       0.75      0.73      0.74      7001
           2       0.74      0.77      0.75      7138

    accuracy                           0.75     14139
   macro avg       0.75      0.75      0.75     14139
weighted avg       0.75      0.75      0.75     14139
              precision    recall  f1-score   support

           0       0.76      0.73      0.74      7067
           2       0.74      0.77      0.75      7072

    accuracy                           0.75     14139
   macro avg       0.75      0.75      0.75     14139
weighted avg       0.75      0.75      0.75     14139
              precision    recall  f1-score   support

           0       0.76      0.73      0.75      7125
           2       0.74      0.76      0.75      7013

    accuracy                           0.75     14138
   macro avg       0.75      0.75      0.75     14138
weighted avg       0.75      0.75      0.75     14138
              precisio

In [5]:
# #SVM
# clf = SVC(C=1.0, kernel='rbf')
# 
# avg_precision = {0: 0, 1: 0, 2: 0}
# avg_recall = {0: 0, 1: 0, 2: 0}
# avg_f1_score = {0: 0, 1: 0, 2: 0}
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
# 
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
# 
#     print(classification_report(y_test, y_pred))
#     # Calculate metrics
#     precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
#     class_to_index = {label: index for index, label in enumerate(sorted(np.unique(y_test)))}
# 
#     # Accumulate metrics for averaging
#     for i in [0, 2]:
#         avg_precision[i] += precision[i]
#         avg_recall[i] += recall[i]
#         avg_f1_score[i] += f1_score[i]
# 
# # Calculate average metrics
# for i in [0, 2]:
#     idx = class_to_index[i]
#     avg_precision[i] /= n_splits
#     avg_recall[i] /= n_splits
#     avg_f1_score[i] /= n_splits
# 
# # Print average metrics
# print("Average Precision:", avg_precision)
# print("Average Recall:", avg_recall)
# print("Average F1-Score:", avg_f1_score)


In [6]:
#KNN
clf = KNeighborsClassifier(n_neighbors=5, algorithm='auto')

avg_precision = {0: 0, 2: 0}
avg_recall = {0: 0, 2: 0}
avg_f1_score = {0: 0, 2: 0}
# Initialize accumulators for the full dataset
avg_precision_full = {0: 0, 2: 0}
avg_recall_full = {0: 0, 2: 0}
avg_f1_score_full = {0: 0, 2: 0}

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    # Calculate metrics
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    class_to_index = {label: index for index, label in enumerate(sorted(np.unique(y_test)))}

    # Accumulate metrics for averaging
    for i in [0, 2]:
        idx = class_to_index[i]
        avg_precision[i] += precision[idx]
        avg_recall[i] += recall[idx]
        avg_f1_score[i] += f1_score[idx]
    #FULL added
    y_pred_full = clf.predict(X_full)
    # Calculate metrics for the full dataset
    precision_full, recall_full, f1_score_full, _ = precision_recall_fscore_support(y_full, y_pred_full, average=None)
    class_to_index_full = {label: index for index, label in enumerate(sorted(np.unique(y_full)))}

    # Accumulate metrics for averaging (full dataset)
    for i in [0, 2]:
        idx_full = class_to_index_full[i]
        avg_precision_full[i] += precision_full[idx_full]
        avg_recall_full[i] += recall_full[idx_full]
        avg_f1_score_full[i] += f1_score_full[idx_full]

# Calculate average metrics
for i in [0, 2]:
    avg_precision[i] /= n_splits
    avg_recall[i] /= n_splits
    avg_f1_score[i] /= n_splits
# Calculate average metrics for the full dataset
for i in [0, 2]:
    avg_precision_full[i] /= n_splits
    avg_recall_full[i] /= n_splits
    avg_f1_score_full[i] /= n_splits
# Print average metrics
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-Score:", avg_f1_score)

# Print average metrics for the full dataset
print("Average Precision on Full Dataset:", avg_precision_full)
print("Average Recall on Full Dataset:", avg_recall_full)
print("Average F1-Score on Full Dataset:", avg_f1_score_full)


              precision    recall  f1-score   support

           0       0.72      0.70      0.71      7001
           2       0.72      0.73      0.72      7138

    accuracy                           0.72     14139
   macro avg       0.72      0.72      0.72     14139
weighted avg       0.72      0.72      0.72     14139
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      7067
           2       0.71      0.73      0.72      7072

    accuracy                           0.71     14139
   macro avg       0.71      0.71      0.71     14139
weighted avg       0.71      0.71      0.71     14139
              precision    recall  f1-score   support

           0       0.73      0.70      0.71      7125
           2       0.70      0.74      0.72      7013

    accuracy                           0.72     14138
   macro avg       0.72      0.72      0.72     14138
weighted avg       0.72      0.72      0.72     14138
              precisio

In [7]:
#Decision Tree
clf = DecisionTreeClassifier(criterion='gini', splitter='best', random_state=42)

avg_precision = {0: 0, 2: 0}
avg_recall = {0: 0, 2: 0}
avg_f1_score = {0: 0,2: 0}
# Initialize accumulators for the full dataset
avg_precision_full = {0: 0, 2: 0}
avg_recall_full = {0: 0, 2: 0}
avg_f1_score_full = {0: 0, 2: 0}

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    # Calculate metrics
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    class_to_index = {label: index for index, label in enumerate(sorted(np.unique(y_test)))}

    # Accumulate metrics for averaging
    for i in [0, 2]:
        idx = class_to_index[i]
        avg_precision[i] += precision[idx]
        avg_recall[i] += recall[idx]
        avg_f1_score[i] += f1_score[idx]
    #FULL added
    y_pred_full = clf.predict(X_full)
    # Calculate metrics for the full dataset
    precision_full, recall_full, f1_score_full, _ = precision_recall_fscore_support(y_full, y_pred_full, average=None)
    class_to_index_full = {label: index for index, label in enumerate(sorted(np.unique(y_full)))}

    # Accumulate metrics for averaging (full dataset)
    for i in [0, 2]:
        idx_full = class_to_index_full[i]
        avg_precision_full[i] += precision_full[idx_full]
        avg_recall_full[i] += recall_full[idx_full]
        avg_f1_score_full[i] += f1_score_full[idx_full]

# Calculate average metrics
for i in [0, 2]:
    avg_precision[i] /= n_splits
    avg_recall[i] /= n_splits
    avg_f1_score[i] /= n_splits
# Calculate average metrics for the full dataset
for i in [0, 2]:
    avg_precision_full[i] /= n_splits
    avg_recall_full[i] /= n_splits
    avg_f1_score_full[i] /= n_splits
# Print average metrics
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-Score:", avg_f1_score)

# Print average metrics for the full dataset
print("Average Precision on Full Dataset:", avg_precision_full)
print("Average Recall on Full Dataset:", avg_recall_full)
print("Average F1-Score on Full Dataset:", avg_f1_score_full)

              precision    recall  f1-score   support

           0       0.66      0.69      0.67      7001
           2       0.68      0.64      0.66      7138

    accuracy                           0.67     14139
   macro avg       0.67      0.67      0.67     14139
weighted avg       0.67      0.67      0.67     14139
              precision    recall  f1-score   support

           0       0.66      0.69      0.68      7067
           2       0.68      0.65      0.66      7072

    accuracy                           0.67     14139
   macro avg       0.67      0.67      0.67     14139
weighted avg       0.67      0.67      0.67     14139
              precision    recall  f1-score   support

           0       0.66      0.68      0.67      7125
           2       0.67      0.64      0.65      7013

    accuracy                           0.66     14138
   macro avg       0.66      0.66      0.66     14138
weighted avg       0.66      0.66      0.66     14138
              precisio

In [8]:
#Gradient Boost
clf = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100)

avg_precision = {0: 0, 2: 0}
avg_recall = {0: 0, 2: 0}
avg_f1_score = {0: 0, 2: 0}
# Initialize accumulators for the full dataset
avg_precision_full = {0: 0, 2: 0}
avg_recall_full = {0: 0, 2: 0}
avg_f1_score_full = {0: 0, 2: 0}

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    # Calculate metrics
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    class_to_index = {label: index for index, label in enumerate(sorted(np.unique(y_test)))}

    # Accumulate metrics for averaging
    for i in [0, 2]:
        idx = class_to_index[i]
        avg_precision[i] += precision[idx]
        avg_recall[i] += recall[idx]
        avg_f1_score[i] += f1_score[idx]
    #FULL added
    y_pred_full = clf.predict(X_full)
    # Calculate metrics for the full dataset
    precision_full, recall_full, f1_score_full, _ = precision_recall_fscore_support(y_full, y_pred_full, average=None)
    class_to_index_full = {label: index for index, label in enumerate(sorted(np.unique(y_full)))}

    # Accumulate metrics for averaging (full dataset)
    for i in [0, 2]:
        idx_full = class_to_index_full[i]
        avg_precision_full[i] += precision_full[idx_full]
        avg_recall_full[i] += recall_full[idx_full]
        avg_f1_score_full[i] += f1_score_full[idx_full]

# Calculate average metrics
for i in [0, 2]:
    avg_precision[i] /= n_splits
    avg_recall[i] /= n_splits
    avg_f1_score[i] /= n_splits
# Calculate average metrics for the full dataset
for i in [0, 2]:
    avg_precision_full[i] /= n_splits
    avg_recall_full[i] /= n_splits
    avg_f1_score_full[i] /= n_splits
# Print average metrics
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-Score:", avg_f1_score)

# Print average metrics for the full dataset
print("Average Precision on Full Dataset:", avg_precision_full)
print("Average Recall on Full Dataset:", avg_recall_full)
print("Average F1-Score on Full Dataset:", avg_f1_score_full)

              precision    recall  f1-score   support

           0       0.77      0.72      0.74      7001
           2       0.74      0.79      0.76      7138

    accuracy                           0.75     14139
   macro avg       0.75      0.75      0.75     14139
weighted avg       0.75      0.75      0.75     14139
              precision    recall  f1-score   support

           0       0.78      0.71      0.74      7067
           2       0.73      0.80      0.76      7072

    accuracy                           0.75     14139
   macro avg       0.75      0.75      0.75     14139
weighted avg       0.75      0.75      0.75     14139
              precision    recall  f1-score   support

           0       0.78      0.71      0.74      7125
           2       0.73      0.79      0.76      7013

    accuracy                           0.75     14138
   macro avg       0.75      0.75      0.75     14138
weighted avg       0.75      0.75      0.75     14138
              precisio

In [9]:
# XGBoost
y[y == 2] = 1
y_full[y_full == 2] = 1

clf = xgb.XGBClassifier(learning_rate=0.1, n_estimators=100, objective='multi:softmax', num_class=2)
avg_precision = {0: 0, 1: 0}
avg_recall = {0: 0, 1: 0}
avg_f1_score = {0: 0, 1: 0}
# Initialize accumulators for the full dataset
avg_precision_full = {0: 0, 1: 0}
avg_recall_full = {0: 0, 1: 0}
avg_f1_score_full = {0: 0, 1: 0}


for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print(classification_report(y_test, y_pred))
    # Calculate metrics
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=1)
    class_to_index = {label: index for index, label in enumerate(sorted(np.unique(y_test)))}
    # Accumulate metrics for averaging
    for i in [0, 1]:
        idx= class_to_index[i]
        avg_precision[i] += precision[idx]
        avg_recall[i] += recall[idx]
        avg_f1_score[i] += f1_score[idx]
    #FULL added
    y_pred_full = clf.predict(X_full)
    # Calculate metrics for the full dataset
    precision_full, recall_full, f1_score_full, _ = precision_recall_fscore_support(y_full, y_pred_full, average=None)
    class_to_index_full = {label: index for index, label in enumerate(sorted(np.unique(y_full)))}
    
    # Accumulate metrics for averaging (full dataset)
    for i in [0, 1]:
        idx_full = class_to_index_full[i]
        avg_precision_full[i] += precision_full[idx_full]
        avg_recall_full[i] += recall_full[idx_full]
        avg_f1_score_full[i] += f1_score_full[idx_full]
    
    
# Calculate average metrics
for i in [0, 1]:
    avg_precision[i] /= n_splits
    avg_recall[i] /= n_splits
    avg_f1_score[i] /= n_splits
# Calculate average metrics for the full dataset
for i in [0, 1]:
    avg_precision_full[i] /= n_splits
    avg_recall_full[i] /= n_splits
    avg_f1_score_full[i] /= n_splits
# Print average metrics
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-Score:", avg_f1_score)

# Print average metrics for the full dataset
print("Average Precision on Full Dataset:", avg_precision_full)
print("Average Recall on Full Dataset:", avg_recall_full)
print("Average F1-Score on Full Dataset:", avg_f1_score_full)

              precision    recall  f1-score   support

           0       0.77      0.71      0.74      7001
           1       0.74      0.79      0.76      7138

    accuracy                           0.75     14139
   macro avg       0.75      0.75      0.75     14139
weighted avg       0.75      0.75      0.75     14139
              precision    recall  f1-score   support

           0       0.78      0.71      0.74      7067
           1       0.73      0.80      0.76      7072

    accuracy                           0.75     14139
   macro avg       0.75      0.75      0.75     14139
weighted avg       0.75      0.75      0.75     14139
              precision    recall  f1-score   support

           0       0.78      0.71      0.74      7125
           1       0.73      0.80      0.76      7013

    accuracy                           0.75     14138
   macro avg       0.75      0.75      0.75     14138
weighted avg       0.75      0.75      0.75     14138
              precisio

In [10]:
# # TensorFlow Neural Network
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from sklearn.metrics import classification_report, precision_recall_fscore_support
# import numpy as np
# 
# # Initialize metrics dictionary for averaging
# avg_precision = {0: 0, 2: 0}
# avg_recall = {0: 0, 2: 0}
# avg_f1_score = {0: 0, 2: 0}
# 
# # Initialize and compile the model
# model = Sequential()
# model.add(Dense(12, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(3, activation='softmax'))
# 
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 
# # Loop for k-fold cross-validation
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     
#     # Fit the model
#     model.fit(X_train, tf.keras.utils.to_categorical(y_train, num_classes=3), epochs=50, batch_size=10, verbose=0)
#     
#     # Predict
#     y_pred = model.predict(X_test)
#     y_pred_mapped = np.argmax(y_pred, axis=1)
#     
#     y_test_mapped = y_test  # Assuming y_test is already a 1-D array
#     
#     # Calculate metrics
#     unique_classes = np.unique(y_test_mapped)
#     precision, recall, f1_score, _ = precision_recall_fscore_support(y_test_mapped, y_pred_mapped, average=None, labels=unique_classes)
#     
#     # Create a mapping from class label to index
#     class_to_index = {label: index for index, label in enumerate(unique_classes)}
#     
#     # Accumulate metrics for averaging
#     for class_label in [0, 2]:
#         idx = class_to_index.get(class_label, None)
#         if idx is not None:
#             avg_precision[class_label] += precision[idx]
#             avg_recall[class_label] += recall[idx]
#             avg_f1_score[class_label] += f1_score[idx]
#         else:
#             avg_precision[class_label] += 0
#             avg_recall[class_label] += 0
#             avg_f1_score[class_label] += 0
#     
# # Calculate average metrics
# n_splits = 5  # Number of splits in k-fold
# for class_label in [0, 2]:
#     avg_precision[class_label] /= n_splits
#     avg_recall[class_label] /= n_splits
#     avg_f1_score[class_label] /= n_splits
# 
# # Print average metrics
# print("Average Precision:", avg_precision)
# print("Average Recall:", avg_recall)
# print("Average F1-Score:", avg_f1_score)
